<a href="https://colab.research.google.com/github/ScientistLim/ProjectAW/blob/feature%2Fmodel%2Flstm-transformer/tft_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pytorch-forecasting
!pip instsall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 38.9 MB/s eta 0:00:00
ERROR: unknown command "instsall" - maybe you meant "install"


In [4]:
%load_ext tensorboard

In [14]:
import pandas as pd
import torch
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import RMSE
from pytorch_lightning import Trainer, LightningModule
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import ReduceLROnPlateau
import matplotlib.pyplot as plt
import datetime
import os

# 엑셀 파일 불러오기
file_path = '/content/drive/MyDrive/종설프1_5팀/전처리.ipynb/four-cycle-16months-feed-pressure-10min.xlsx'
df = pd.read_excel(file_path)

# 데이터 전처리: 필요한 컬럼 설정 (Time, feed_pressure)
df['time'] = pd.to_datetime(df['Time'])
df['time_idx'] = ((df['time'] - df['time'].min()).dt.total_seconds() // 600).astype(int)  # 5분 간격의 시간 인덱스 생성
df['group'] = "feed_pressure"  # 그룹화 컬럼 추가

# 데이터 확인
print(df.head())

# 데이터 길이 확인
total_length = len(df)
print(f"Total data length: {total_length} rows")

                 Time  feed_pressure                time  time_idx  \
0 2023-07-09 04:17:00       0.000000 2023-07-09 04:17:00         0   
1 2023-07-09 04:27:00       0.048376 2023-07-09 04:27:00         1   
2 2023-07-09 04:37:00       0.086913 2023-07-09 04:37:00         2   
3 2023-07-09 04:47:00       0.110682 2023-07-09 04:47:00         3   
4 2023-07-09 04:57:00       0.306872 2023-07-09 04:57:00         4   

           group  
0  feed_pressure  
1  feed_pressure  
2  feed_pressure  
3  feed_pressure  
4  feed_pressure  
Total data length: 64775 rows


### 데이터 준비 및 설정
우선 max_encoder_length와 max_prediction_length를 설정하여 데이터의 인코더 및 예측 길이를 정의합니다.

In [15]:
# max_encoder_length와 max_prediction_length 설정
max_encoder_length = 5000  # 30000 - 약 104일 (10분 간격의 데이터)
max_prediction_length = 500  # 3000 - 약 10일 예측

print(f"Max encoder length: {max_encoder_length} entries")
print(f"Max prediction length: {max_prediction_length} entries")

Max encoder length: 5000 entries
Max prediction length: 500 entries


### TimeSeriesDataSet 생성
TimeSeriesDataSet을 사용해 데이터를 생성합니다. 여기에는 타겟, 그룹화 방식, 그리고 변동하는 시계열 컬럼을 정의합니다.


In [16]:
# TimeSeriesDataSet 생성
training = TimeSeriesDataSet(
    df,
    time_idx="time_idx",
    target="feed_pressure",
    group_ids=["group"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_unknown_reals=["feed_pressure"],
    time_varying_known_reals=["time_idx"],
    target_normalizer=GroupNormalizer(groups=["group"], transformation="softplus"),
    allow_missing_timesteps=True
)

### DataLoader 생성
DataLoader를 생성하여 모델이 데이터를 학습할 수 있도록 준비합니다.

In [9]:
# DataLoader 생성
batch_size = 8
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)

### Temporal Fusion Transformer 모델 생성
Temporal Fusion Transformer 모델을 생성하여 데이터셋을 학습할 준비를 합니다.

In [10]:
# Temporal Fusion Transformer 모델 생성
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=8,  # 줄여서 사용해 보기
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=8,
    loss=RMSE(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
/usr/local/lib/python3.10/dist-packages/pytorch_forecasting/models/temporal_fusion_transformer/__init__.py:143: UserWarning: In pytorch-forecasting models, on versions 1.1.X, the default optimizer defaults to 'adam', if pytorch_optimizer is not installed, otherwise it defaults to 'ranger' from pytorch_optimizer. From version 1.2.0, the default optimizer will be 'adam' regardless of whether pytorch_optimizer is installed, in order to minimize the number of 

### LightningModule 정의 및 Trainer 설정
PyTorch Lightning을 이용하여 모델 학습을 보다 쉽게 관리할 수 있도록 합니다.


In [ ]:
# LightningModule 정의
class TFTLightningModule(LightningModule):
    def __init__(self, tft_model):
        super().__init__()
        self.tft_model = tft_model.cuda()

    def forward(self, x):
        return self.tft_model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        x = {key: val.cuda().contiguous() for key, val in x.items()}
        y = y[0].cuda().contiguous() if isinstance(y, tuple) else y.cuda().contiguous()
        y_hat = self(x)
        loss = self.tft_model.loss(y_hat["prediction"], y)
        self.log("train_loss", loss, batch_size=len(x['encoder_cont'].squeeze()))
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = {key: val.cuda().contiguous() for key, val in x.items()}
        y = y[0].cuda().contiguous() if isinstance(y, tuple) else y.cuda().contiguous()
        y_hat = self(x)
        loss = self.tft_model.loss(y_hat["prediction"], y)
        self.log("val_loss", loss, batch_size=len(x['encoder_cont'].squeeze()))
        return loss

    def configure_optimizers(self):
      optimizer = torch.optim.Adam(self.tft_model.parameters(), lr=0.03)
      scheduler = {
        'scheduler': ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=1e-6, verbose=True),
        'monitor': 'val_loss'
        }
        return [optimizer], [scheduler]

In [11]:
# TFT 모델을 LightningModule로 감싸기
tft_module = TFTLightningModule(tft)

# 학습을 위한 EarlyStopping 설정
early_stop_callback = EarlyStopping(monitor="val_loss", patience=10, min_delta=1e-4)

# 체크포인트 설정
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/",
    filename="tft-best-checkpoint",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

# Learning Rate 스케줄러 설정
lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=True,
    min_lr=1e-6
)

# Trainer 설정
trainer = Trainer(
    max_epochs=50,
    accelerator='gpu',
    devices=1,
    gradient_clip_val=0.1,
    callbacks=[early_stop_callback, checkpoint_callback, lr_scheduler],
    accumulate_grad_batches=2,  # 배치를 2번 누적
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


### 모델 시각화

In [30]:
import torch

# 모델을 GPU로 이동한 후 요약 정보를 출력합니다.
tft_module.cuda()

# 샘플 입력 준비
sample_input = next(iter(training.to_dataloader(train=False, batch_size=1, num_workers=0)))[0]
sample_input = {key: val.cuda() for key, val in sample_input.items()}

# 모델 구조를 요약하기 위해 사용자 정의 함수 작성
def model_summary(model):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"\nModel Summary:")
    print(f"Total Trainable Parameters: {total_params}")
    print("Layers and Parameter Sizes:")
    for name, param in model.named_parameters():
        if param.requires_grad:
            print(f"{name}: {list(param.size())}")

# 사용자 정의 요약 함수 호출
model_summary(tft_module)



Model Summary:
Total Trainable Parameters: 4831
Layers and Parameter Sizes:
tft_model.prescalers.time_idx.weight: [8, 1]
tft_model.prescalers.time_idx.bias: [8]
tft_model.prescalers.feed_pressure.weight: [8, 1]
tft_model.prescalers.feed_pressure.bias: [8]
tft_model.encoder_variable_selection.flattened_grn.resample_norm.mask: [2]
tft_model.encoder_variable_selection.flattened_grn.resample_norm.norm.weight: [2]
tft_model.encoder_variable_selection.flattened_grn.resample_norm.norm.bias: [2]
tft_model.encoder_variable_selection.flattened_grn.fc1.weight: [2, 16]
tft_model.encoder_variable_selection.flattened_grn.fc1.bias: [2]
tft_model.encoder_variable_selection.flattened_grn.context.weight: [2, 8]
tft_model.encoder_variable_selection.flattened_grn.fc2.weight: [2, 2]
tft_model.encoder_variable_selection.flattened_grn.fc2.bias: [2]
tft_model.encoder_variable_selection.flattened_grn.gate_norm.glu.fc.weight: [4, 2]
tft_model.encoder_variable_selection.flattened_grn.gate_norm.glu.fc.bias: [4]


In [32]:
# 모델 학습
trainer.fit(tft_module, train_dataloader)

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type                      | Params | Mode 
----------------------------------------------------------------
0 | tft_model | TemporalFusionTransformer | 4.8 K  | train
----------------------------------------------------------------
4.8 K     Trainable params
0         Non-trainable params
4.8 K     Total params
0.019     Total estimated model params size (MB)
159       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

RuntimeError: Early stopping conditioned on metric `val_loss` which is not available. Pass in or modify your `EarlyStopping` callback to use any of the following: `train_loss`

In [33]:
# 6개월 예측
encoder_data = df[df['time_idx'] > df['time_idx'].max() - max_encoder_length]
new_prediction_data = pd.DataFrame({
    "Time": pd.date_range(df["Time"].max() + pd.Timedelta(minutes=10), periods=max_prediction_length, freq="10T"),
    "feed_pressure": [0] * max_prediction_length,
    "time_idx": range(df["time_idx"].max() + 1, df["time_idx"].max() + 1 + max_prediction_length),
    "group": "feed_pressure",
})

new_data = pd.concat([encoder_data, new_prediction_data]).reset_index(drop=True)

# 예측 수행
new_data_cuda = training.transform(new_data)
new_data_cuda = {key: val.cuda() for key, val in new_data_cuda.items()}
predictions = tft_module.tft_model.predict(new_data_cuda)

<ipython-input-33-130e958a5e3e>:4: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  "Time": pd.date_range(df["Time"].max() + pd.Timedelta(minutes=10), periods=max_prediction_length, freq="10T"),


AttributeError: 'TimeSeriesDataSet' object has no attribute 'transform'

In [ ]:
# 예측 결과 시각화
plt.figure(figsize=(10, 6))
plt.plot(df['Time'], df['feed_pressure'], label='Historical Feed Pressure', color='blue')
plt.plot(new_prediction_data['Time'], predictions, label='Predicted Feed Pressure (6 months)', color='red')
plt.xlabel('Time')
plt.ylabel('Feed Pressure')
plt.legend()
plt.title('TFT Model Feed Pressure Prediction for Next 6 Months')
plt.show()